In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import implicit

In [4]:
len(list(range(1523, 16753+1523, 1523)))

11

In [5]:
#считывание данных
users = pd.read_csv('../../data/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv('../../data/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv('../../data/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

In [6]:
print(f"Кол-во пользователей: {len(train_transactions['chb'].unique())}")
print(f"Кол-во документов в истории пользователей: {len(train_transactions['sys_numb'].unique())}")
print(f"Общее кол-во документов: {len(items['sys_numb'].unique())}")

Кол-во пользователей: 16753
Кол-во документов в истории пользователей: 194666
Общее кол-во документов: 354355


In [7]:
#строго фиксируем кол-во пользователей и уникальных документов
n_users = len(train_transactions['chb'].unique())
n_items = len(items['sys_numb'].unique())

In [8]:
#т.к далее придётся работать с матрицами созданим словари, которые точно отображают индексы в chb/sys_numb и обратно
mapping_chb_index = {chb_number: index for index, chb_number in enumerate(train_transactions['chb'].unique())}
mapping_sys_numb_index = {sys_number: index for index, sys_number in enumerate(items['sys_numb'].unique())}

mapping_index_chb = {index: chb_number for index, chb_number in enumerate(train_transactions['chb'].unique())}
mapping_index_sys_numb = {index: sys_number for index, sys_number in enumerate(items['sys_numb'].unique())}

In [9]:
#в базовом решении будем не будем использовать дополнительные данные о взаимодействиях
train_transactions = train_transactions[['chb', 'sys_numb']]

In [10]:
from sklearn.model_selection import train_test_split

# делим данные на тренировочный и тестовый наборы
train_data, test_data = train_test_split(train_transactions, test_size=0.20)

In [11]:
# Не очень удачное разбиение на train и test, поскольку в выборке для тестирования присутствуют не все пользователи. 
# Это означает, что понять, насколько рекомендательна система качественно работает для данного сегмента не получится.

print(f"Кол-во уникальных пользователей: {len(train_transactions['chb'].unique())}")
print(f"Кол-во уникальных пользвоателей в выборке для обучения: {len(train_data['chb'].unique())}")
print(f"Кол-во уникальных пользвоателей в выборке для тестирования: {len(test_data['chb'].unique())}")

Кол-во уникальных пользователей: 16753
Кол-во уникальных пользвоателей в выборке для обучения: 16660
Кол-во уникальных пользвоателей в выборке для тестирования: 12371


In [12]:
# Имеем дело с разряженными матрицами с ними лучше работать в sparse формате

def df_to_sparse(df):
  row = []
  col = []
  data = []

  for line in df.itertuples():
    row.append(mapping_chb_index[line.chb])
    col.append(mapping_sys_numb_index[line.sys_numb])
    data.append(1)

  return csr_matrix((data, (row, col)))

In [13]:
# получение sparse матрицы user-item для train/test
train_data_sparse = df_to_sparse(train_data)
test_data_sparse = df_to_sparse(test_data)

In [2]:
from implicit.als import AlternatingLeastSquares

#В качестве базового решения попробуем алгоритм ALS. Подробнее можно ознакомиться с ним здесь - https://github.com/benfred/implicit
model = AlternatingLeastSquares(num_threads=32)
model.fit(train_data_sparse)

NameError: name 'train_data_sparse' is not defined

In [1]:
# Получим рекомендации для конкретного пользователя 
userid = 2233
ids, scores = model.recommend(userid, train_data_sparse[userid], N=20, filter_already_liked_items=False)

NameError: name 'model' is not defined

In [ ]:
# Отобразим рекомендации в DataFrame
top20recom_df = pd.DataFrame({"sys_numb": [mapping_index_sys_numb[id] for id in ids], "score": scores, "already_liked": np.in1d(ids, train_data_sparse[userid].indices)})

In [ ]:
def get_recom(userid):
  ids, scores = model.recommend(userid, train_data_sparse[userid], N=20, filter_already_liked_items=False)
  top20recom_df = pd.DataFrame({"sys_numb": [mapping_index_sys_numb[id] for id in ids], "score": scores, "already_liked": np.in1d(ids, train_data_sparse[userid].indices)})
  return top20recom_df['sys_numb'].values

In [ ]:
#подбор рекомендаций для всех пользователей из train

all_rec = []

for userid in tqdm(range(train_data_sparse.shape[0])):
  user_chb = mapping_index_chb[userid]
  user_rec = get_recom(userid)
  for rec in user_rec:
    all_rec.append([user_chb, rec])

In [ ]:
# DataFrame для отправки решения с рекомендациями
solution = pd.DataFrame(all_rec, columns=["chb", "sys_numb"])

In [ ]:
# Формирование csv файла для отправки на платформу
solution.to_csv("solution.csv", index=False, sep=';')

## Метрика

In [ ]:
df_solution = solution
df_grd = test_data

In [ ]:
#считаем recall, precision, f1_score

def metric(df_solution, df_grd):
  pred = set(df_solution['chb'] + '_' + df_solution['sys_numb'].values)
  true = set(df_grd['chb'] + '_' + df_grd['sys_numb'].values)
  recall = len(pred.intersection(true)) / len(true)
  precision = len(pred.intersection(true)) / (20 * len(df_grd['chb'].unique()))
  f1_score = 2 * (precision * recall) / (precision + recall)
  print(f"Recall: {round(recall, 5)}")
  print(f"Precision: {round(precision, 5)}")
  print(f"F1-score: {round(f1_score, 5)}")

In [ ]:
# оценка базового решения

metric(df_solution, df_grd)

In [ ]:
# подадим ответы в качестве рекомендаций

metric(df_grd, df_grd)

In [ ]:
# создадим случайные рекомендации 
random_solution = []
for chb in tqdm(set(df_grd['chb'].values)):
  for sys in items['sys_numb'].sample(20).values:
    random_solution.append([chb, sys])

In [ ]:
# оценка подхода со случайными рекомендациями
metric(pd.DataFrame(random_solution, columns=['chb', 'sys_numb']), df_grd)

In [ ]:
from typing import Any

def get_something(x, y) -> tuple[float, Any]:

  return x, y

get_something()